In [1]:
%matplotlib ipympl
from impedance.preprocessing import readBioLogic
from impedance.models.circuits import CustomCircuit
import matplotlib.pyplot as plt
import numpy as np
from models import *
import mpmath as mp
import os
import csv

In [2]:
folder_path=os.path.join("..","exp data","single type")
for file_name in os.listdir(folder_path):
    file_path=os.path.join(folder_path,file_name)
    f,z=readBioLogic(file_path)

    Rsep=100
    R1=1000
    Q=1e-4
    alpha=1
    c=CustomCircuit(initial_guess=[Rsep,R1,Q,alpha],circuit="R_1-TLMuni")

    mp.dps=300
    c.fit(impedance=z,frequencies=f,bounds=((0,0,0,0.9),(np.inf,np.inf,np.inf,1)))
    
    z_fitted=c.predict(frequencies=f)
    data=[
        ["filename","Rsep","R1","Q","α"], 
        [f"{file_name}",c.parameters_[0],c.parameters_[1],c.parameters_[2],c.parameters_[3]]
    ]
    fcsv="../results/single.csv"
    with open(fcsv,"a",newline="",encoding='UTF-8') as csv_file:
        writer=csv.writer(csv_file)
        writer.writerows(data)

e:\code\github.com\yugal0\fitting-experimental\code\models.py:13: RuntimeWarning: overflow encountered in tanh
  Z= 2*np.sqrt(R1/((omega*1j)**alpha*Q))*(1/(np.tanh(np.sqrt((1j*omega)**alpha*R1*Q))))
